In [1]:
from keras.applications import vgg16
model = vgg16.VGG16(weights=None, input_shape=[201, 201, 2])

Using TensorFlow backend.
C:\Users\tomni\Anaconda3\envs\keras-gpu\lib\site-packages\keras_applications\imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  str(input_shape[-1]) + ' input channels.')
W1029 12:52:41.806256  8184 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1029 12:52:41.886043  8184 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1029 12:52:41.910979  8184 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\keras-gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. P

In [8]:
# https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
    
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(" — val_f1: %f — val_precision: %f — val_recall %f" % (_val_f1, _val_precision, _val_recall))
        return
 
metrics = Metrics()

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
from dataGenerator import DataGenerator
from personal_data import location_data
dataGenerator = DataGenerator(location_data, batch_size=128)

In [12]:
from keras import callbacks
tensorboard = callbacks.TensorBoard(log_dir='tensorboard')

In [15]:
model.fit_generator(dataGenerator.generate(), epochs=1)

ValueError: `steps_per_epoch=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps_per_epoch` or use the `keras.utils.Sequence` class.